In [2]:
import numpy as np
import sys
import torch
import os

from lib.features import init_backbone
from lib.protopnet.model import PPNet
from src.utils.argument_parser import read_from_file

In [22]:
def init_model(args_path="default_args.yaml", model_type="teacher", checkpoint_path=None, save_pkl=False, output_path="model.pkl"):
    """
    initializes a protopnet model from a checkpoint path
    args_path: path to file with arguments about the model (default args will work for cars dataset and ppnet with resnet50 or 18)
    model_type: "teacher" or "student"
    checkpoint_path: file path to checkpoint
    save_pkl: whether to save the model as a.pkl
    output_path: only applies if saving model as pkl
    """

    args = read_from_file(args_path)[0]
    print(args)

    if model_type == "teacher":
        backbone = args.settingsConfig.Teacherbackbone
    elif model_type == "student":
        backbone = args.settingsConfig.Studentbackbone
    else:
        print("this model type is not supported") 
        return

    use_chkpt_opt = args.settingsConfig.useCheckpointOptimizer

    features, trainable_param_names = init_backbone(backbone)

    model = PPNet(num_classes=NUM_CLASSES, feature_net=features, args=args.settingsConfig)

    if checkpoint_path is not None:
        checkpoint = torch.load(checkpoint_path) 
        model = checkpoint['model']
        print("Loaded model from ", checkpoint_path)

        if not use_chkpt_opt:
            checkpoint = None
    else:
        checkpoint = None

    if args.common.mgpus:
        print("Multi-gpu setting")
        model = torch.nn.DataParallel(model)

    if args.common.cuda > 0:
        print("Using GPU")
        model.cuda()

    if save_pkl:
        torch.save(model, output_path)
        print(f"Model saved at {output_path}")

    return model

In [23]:
NUM_CLASSES = 196
model = init_model(checkpoint_path="../saved_models/teacher_protopnet_22_0.7737.pth", save_pkl=True)

{'dataConfig': {'imgSize': 224, 'loaderName': 'cars', 'augment': False}, 'settingsConfig': {'imgSize': 224, 'numFeatures': 256, 'prototypicalModel': 'protopnet', 'PrototypeActivationFunction': 'log', 'useCheckpointOptimizer': True, 'Teacherbackbone': {'prototypicalModel': 'protopnet', 'name': 'resnet50', 'pretrained': True}, 'Studentbackbone': {'prototypicalModel': 'protopnet', 'name': 'resnet18', 'pretrained': True}, 'protoPNet': {'numPrototypes': 1960, 'classSpecific': True, 'W1': 1, 'H1': 1}, 'train': {'seed': 12}}, 'common': {'cuda': True, 'mgpus': True, 'numWorkers': 0, 'testBatchSize': 1, 'testNumWorkers': 0, 'trainBatchSize': 4, 'trainNumWorkers': 1}}
Loaded model from  ../saved_models/teacher_protopnet_22_0.7737.pth
Multi-gpu setting
Using GPU
Model saved at model.pkl
